# [Hands-On] 텍스트 전처리 파이프라인 구축

- Author: Sangkeun Jung (hugmanskj@gmail.com)

> 교육 목적

**Copyright**: All rights reserved

---

## 개요

텍스트 데이터의 전처리 파이프라인을 구축하는 실습입니다.
불용어 제거, 표준화, 정규화 기법을 학습하고 의료 판독문에 적용합니다.


**학습 목표**:
1. 불용어(Stopwords) 제거 방법 이해 및 구현
2. 표준화(Normalization) 기법 습득
3. 정규화(Regularization) 기법 습득
4. 전처리 파이프라인 통합 구축

---

## 1. 라이브러리 임포트

In [ ]:
import re
import numpy as np
from collections import Counter

---

## 2. 불용어(Stopwords) 제거

### 불용어란?
- 텍스트 분석에서 의미가 없거나 중요하지 않은 단어
- 한국어: 조사, 어미, 접속사 등
- 영어: a, an, the, is, are 등

### 불용어 제거의 효과
- Vocabulary 크기 감소 (30-50% 감소)
- 계산 효율성 증가
- 의미 있는 단어에 집중

In [ ]:
# 한국어 불용어 리스트 (예시)
KOREAN_STOPWORDS = set([
    # 조사
    '이', '가', '을', '를', '은', '는', '에', '에서', '으로', '로',
    '의', '와', '과', '도', '만', '부터', '까지', '께서', '에게', '한테',

    # 어미
    '다', '이다', '입니다', '습니다', '아', '어', '었', '였', '네', '요',

    # 접속사
    '그리고', '하지만', '그러나', '그래서', '또한', '또는', '및',

    # 부사
    '매우', '아주', '너무', '정말', '진짜', '완전', '조금', '약간', '많이',

    # 대명사
    '이것', '그것', '저것', '여기', '거기', '저기', '이곳', '그곳', '저곳',

    # 기타
    '것', '수', '등', '때', '중', '내', '간'
])

In [ ]:
# 영어 불용어 리스트 (예시)
ENGLISH_STOPWORDS = set([
    'a', 'an', 'the', 'and', 'or', 'but', 'if', 'then', 'else',
    'is', 'are', 'was', 'were', 'be', 'been', 'being',
    'have', 'has', 'had', 'do', 'does', 'did',
    'i', 'you', 'he', 'she', 'it', 'we', 'they',
    'my', 'your', 'his', 'her', 'its', 'our', 'their',
    'this', 'that', 'these', 'those',
    'in', 'on', 'at', 'to', 'for', 'of', 'with', 'by', 'from',
    'very', 'too', 'so', 'much', 'many', 'more', 'most'
])

### 불용어 제거 함수 구현

In [ ]:
def remove_stopwords(tokens, stopwords):
    """
    토큰 리스트에서 불용어를 제거하는 함수

    Parameters:
    -----------
    tokens : list of str
        토큰 리스트
    stopwords : set of str
        불용어 집합

    Returns:
    --------
    list of str
        불용어가 제거된 토큰 리스트
    """
    filtered_tokens = [token for token in tokens if token not in stopwords]
    return filtered_tokens

### 예제 1: 한국어 불용어 제거

In [ ]:
print("=" * 80)
print("[예제 1] 한국어 불용어 제거")
print("=" * 80)

# 예제 문장
sentence1 = "나는 오늘 아주 매우 많이 피곤했다"
tokens1 = sentence1.split()

print("\n원본 문장:")
print("  \"%s\"" % sentence1)
print("\n토큰 리스트:")
print("  %s" % tokens1)

# 불용어 제거
filtered1 = remove_stopwords(tokens1, KOREAN_STOPWORDS)

print("\n불용어 제거 후:")
print("  %s" % filtered1)
print("\n제거된 불용어:")
print("  %s" % [t for t in tokens1 if t not in filtered1])
print("\n감소율: %.1f%% (%d → %d 토큰)" % (
    100.0 * (len(tokens1) - len(filtered1)) / len(tokens1),
    len(tokens1),
    len(filtered1)
))

[예제 1] 한국어 불용어 제거

원본 문장:
  "나는 오늘 아주 매우 많이 피곤했다"

토큰 리스트:
  ['나는', '오늘', '아주', '매우', '많이', '피곤했다']

불용어 제거 후:
  ['나는', '오늘', '피곤했다']

제거된 불용어:
  ['아주', '매우', '많이']

감소율: 50.0% (6 → 3 토큰)


In [ ]:
# 예제 문장 2
sentence2 = "비가 많이 내렸다 그래서 나는 우산을 가지고 나갔다"
tokens2 = sentence2.split()

print("\n" + "=" * 80)
print("원본 문장:")
print("  \"%s\"" % sentence2)
print("\n토큰 리스트:")
print("  %s" % tokens2)

filtered2 = remove_stopwords(tokens2, KOREAN_STOPWORDS)

print("\n불용어 제거 후:")
print("  %s" % filtered2)
print("\n제거된 불용어:")
print("  %s" % [t for t in tokens2 if t not in filtered2])
print("\n감소율: %.1f%% (%d → %d 토큰)" % (
    100.0 * (len(tokens2) - len(filtered2)) / len(tokens2),
    len(tokens2),
    len(filtered2)
))


원본 문장:
  "비가 많이 내렸다 그래서 나는 우산을 가지고 나갔다"

토큰 리스트:
  ['비가', '많이', '내렸다', '그래서', '나는', '우산을', '가지고', '나갔다']

불용어 제거 후:
  ['비가', '내렸다', '나는', '우산을', '가지고', '나갔다']

제거된 불용어:
  ['많이', '그래서']

감소율: 25.0% (8 → 6 토큰)


### 예제 2: 영어 불용어 제거

In [ ]:
print("\n" + "=" * 80)
print("[예제 2] 영어 불용어 제거")
print("=" * 80)

sentence_en = "The patient has a mild cough and fever but no chest pain"
tokens_en = sentence_en.lower().split()

print("\n원본 문장:")
print("  \"%s\"" % sentence_en)
print("\n토큰 리스트 (소문자):")
print("  %s" % tokens_en)

filtered_en = remove_stopwords(tokens_en, ENGLISH_STOPWORDS)

print("\n불용어 제거 후:")
print("  %s" % filtered_en)
print("\n제거된 불용어:")
print("  %s" % [t for t in tokens_en if t not in filtered_en])
print("\n감소율: %.1f%% (%d → %d 토큰)" % (
    100.0 * (len(tokens_en) - len(filtered_en)) / len(tokens_en),
    len(tokens_en),
    len(filtered_en)
))


[예제 2] 영어 불용어 제거

원본 문장:
  "The patient has a mild cough and fever but no chest pain"

토큰 리스트 (소문자):
  ['the', 'patient', 'has', 'a', 'mild', 'cough', 'and', 'fever', 'but', 'no', 'chest', 'pain']

불용어 제거 후:
  ['patient', 'mild', 'cough', 'fever', 'no', 'chest', 'pain']

제거된 불용어:
  ['the', 'has', 'a', 'and', 'but']

감소율: 41.7% (12 → 7 토큰)


---

## 3. 표준화(Normalization)

### 표준화란?
- 동일한 의미를 가진 다양한 형태의 텍스트를 통일된 형태로 변환
- 예: "좋아요", "좋아아아요", "좋아아아아아요" → "좋아요"

### 표준화 기법
1. 대소문자 통일 (영어)
2. 반복 문자 축약
3. 특수문자 처리
4. 공백 정규화

### 표준화 함수 구현

In [ ]:
def normalize_text(text):
    """
    텍스트를 표준화하는 함수

    Parameters:
    -----------
    text : str
        원본 텍스트

    Returns:
    --------
    str
        표준화된 텍스트
    """
    # 1. 소문자 변환 (영어)
    text = text.lower()

    # 2. 반복되는 문자 축약 (3개 이상 → 1개)
    # 예: "좋아아아아요" → "좋아요"
    text = re.sub(r'(.)\1{2,}', r'\1', text)

    # 3. 반복되는 구두점 축약 (2개 이상 → 1개)
    # 예: "!!!!" → "!"
    text = re.sub(r'([!?.])\1+', r'\1', text)

    # 4. 다중 공백을 단일 공백으로
    text = re.sub(r'\s+', ' ', text)

    # 5. 앞뒤 공백 제거
    text = text.strip()

    return text

### 예제 3: 표준화 적용

In [ ]:
print("\n" + "=" * 80)
print("[예제 3] 텍스트 표준화")
print("=" * 80)

# 예제 텍스트들
test_texts = [
    "좋아아아아요!!! 정말 최고예요!!!",
    "ㅋㅋㅋㅋㅋㅋㅋㅋ 진짜    너무   웃겨요  ㅎㅎㅎ",
    "HELLO World!!!   HOW    ARE   YOU???",
    "대박!!!!!! 완전 신기해요오오오오"
]

for i, text in enumerate(test_texts, 1):
    normalized = normalize_text(text)
    print("\n[%d] 원본:" % i)
    print("    \"%s\"" % text)
    print("    표준화:")
    print("    \"%s\"" % normalized)
    print("    길이: %d → %d (%.1f%% 감소)" % (
        len(text),
        len(normalized),
        100.0 * (len(text) - len(normalized)) / len(text)
    ))


[예제 3] 텍스트 표준화

[1] 원본:
    "좋아아아아요!!! 정말 최고예요!!!"
    표준화:
    "좋아요! 정말 최고예요!"
    길이: 20 → 13 (35.0% 감소)

[2] 원본:
    "ㅋㅋㅋㅋㅋㅋㅋㅋ 진짜    너무   웃겨요  ㅎㅎㅎ"
    표준화:
    "ㅋ 진짜 너무 웃겨요 ㅎ"
    길이: 28 → 13 (53.6% 감소)

[3] 원본:
    "HELLO World!!!   HOW    ARE   YOU???"
    표준화:
    "hello world! how are you?"
    길이: 36 → 25 (30.6% 감소)

[4] 원본:
    "대박!!!!!! 완전 신기해요오오오오"
    표준화:
    "대박! 완전 신기해요오"
    길이: 20 → 12 (40.0% 감소)


---

## 4. 정규화(Regularization)

### 정규화란?
- 특정 패턴을 가진 텍스트를 대표 토큰으로 치환
- 예: "2024년 12월 1일" → "DATE"
- 예: "010-1234-5678" → "PHONE"
- 예: "12,000원" → "NUM원"

### 정규화의 장점
- Vocabulary 크기 대폭 감소
- 일반화 성능 향상
- 숫자/날짜/전화번호 등의 의미 보존

### 정규화 함수 구현

In [ ]:
def normalize_numbers(text):
    """
    숫자를 NUM 토큰으로 치환

    Parameters:
    -----------
    text : str
        원본 텍스트

    Returns:
    --------
    str
        숫자가 정규화된 텍스트
    """
    # 천 단위 구분 쉼표가 있는 숫자 (예: 12,000)
    text = re.sub(r'\d{1,3}(,\d{3})+', 'NUM', text)

    # 소수점 숫자 (예: 3.14, 98.6)
    text = re.sub(r'\d+\.\d+', 'NUM', text)

    # 일반 정수 (예: 123, 45)
    text = re.sub(r'\d+', 'NUM', text)

    return text

In [ ]:
def normalize_dates(text):
    """
    날짜를 DATE 토큰으로 치환

    Parameters:
    -----------
    text : str
        원본 텍스트

    Returns:
    --------
    str
        날짜가 정규화된 텍스트
    """
    # 패턴 1: YYYY-MM-DD, YYYY/MM/DD, YYYY.MM.DD
    text = re.sub(r'\d{4}[-/.]\d{1,2}[-/.]\d{1,2}', 'DATE', text)

    # 패턴 2: YYYY년MM월DD일
    text = re.sub(r'\d{4}년\s?\d{1,2}월\s?\d{1,2}일', 'DATE', text)

    # 패턴 3: MM/DD/YYYY (미국식)
    text = re.sub(r'\d{1,2}/\d{1,2}/\d{4}', 'DATE', text)

    return text

In [ ]:
def normalize_phone_numbers(text):
    """
    전화번호를 PHONE 토큰으로 치환

    Parameters:
    -----------
    text : str
        원본 텍스트

    Returns:
    --------
    str
        전화번호가 정규화된 텍스트
    """
    # 패턴 1: XXX-XXXX-XXXX
    text = re.sub(r'\d{2,3}-\d{3,4}-\d{4}', 'PHONE', text)

    # 패턴 2: XXXXXXXXXXX (연속된 숫자)
    text = re.sub(r'\d{10,11}', 'PHONE', text)

    return text

In [ ]:
def normalize_patterns(text):
    """
    모든 패턴을 정규화하는 통합 함수

    Parameters:
    -----------
    text : str
        원본 텍스트

    Returns:
    --------
    str
        패턴이 정규화된 텍스트
    """
    # 순서 중요: 전화번호 → 날짜 → 숫자
    text = normalize_phone_numbers(text)
    text = normalize_dates(text)
    text = normalize_numbers(text)

    return text

### 예제 4: 정규화 적용

In [ ]:
print("\n" + "=" * 80)
print("[예제 4] 패턴 정규화")
print("=" * 80)

test_cases = [
    "2024년12월1일에 010-1234-5678로 연락주세요.",
    "가격은 12,000원이고 무게는 3.5kg입니다.",
    "예약 날짜: 2024-12-15, 전화: 02-1234-5678",
    "검사 비용: 150,000원 (2024.12.01 기준)"
]

for i, text in enumerate(test_cases, 1):
    normalized = normalize_patterns(text)
    print("\n[%d] 원본:" % i)
    print("    \"%s\"" % text)
    print("    정규화:")
    print("    \"%s\"" % normalized)


[예제 4] 패턴 정규화

[1] 원본:
    "2024년12월1일에 010-1234-5678로 연락주세요."
    정규화:
    "DATE에 PHONE로 연락주세요."

[2] 원본:
    "가격은 12,000원이고 무게는 3.5kg입니다."
    정규화:
    "가격은 NUM원이고 무게는 NUMkg입니다."

[3] 원본:
    "예약 날짜: 2024-12-15, 전화: 02-1234-5678"
    정규화:
    "예약 날짜: DATE, 전화: PHONE"

[4] 원본:
    "검사 비용: 150,000원 (2024.12.01 기준)"
    정규화:
    "검사 비용: NUM원 (DATE 기준)"


---

## 5. 통합 전처리 파이프라인

### TextPreprocessor 클래스
- 모든 전처리 단계를 하나의 파이프라인으로 통합
- 각 단계를 선택적으로 적용 가능

In [ ]:
class TextPreprocessor:
    """
    텍스트 전처리 파이프라인 클래스
    """

    def __init__(self,
                 stopwords=None,
                 remove_stopwords_flag=True,
                 normalize_flag=True,
                 regularize_flag=True):
        """
        Parameters:
        -----------
        stopwords : set of str
            불용어 집합
        remove_stopwords_flag : bool
            불용어 제거 여부
        normalize_flag : bool
            표준화 적용 여부
        regularize_flag : bool
            정규화 적용 여부
        """
        self.stopwords = stopwords if stopwords is not None else set()
        self.remove_stopwords_flag = remove_stopwords_flag
        self.normalize_flag = normalize_flag
        self.regularize_flag = regularize_flag

    def preprocess(self, text):
        """
        텍스트를 전처리하는 메인 함수

        Parameters:
        -----------
        text : str
            원본 텍스트

        Returns:
        --------
        list of str
            전처리된 토큰 리스트
        """
        # 1. 정규화 (패턴 치환)
        if self.regularize_flag:
            text = normalize_patterns(text)

        # 2. 표준화 (대소문자, 반복 문자 등)
        if self.normalize_flag:
            text = normalize_text(text)

        # 3. 토큰화 (공백 기준)
        tokens = text.split()

        # 4. 불용어 제거
        if self.remove_stopwords_flag:
            tokens = remove_stopwords(tokens, self.stopwords)

        return tokens

    def preprocess_full(self, text):
        """
        전처리 전후 비교를 위한 함수

        Returns:
        --------
        dict
            원본 텍스트, 각 단계별 결과, 최종 결과
        """
        results = {
            'original': text,
            'regularized': text,
            'normalized': text,
            'tokenized': [],
            'filtered': []
        }

        # 1. 정규화
        if self.regularize_flag:
            results['regularized'] = normalize_patterns(text)

        # 2. 표준화
        if self.normalize_flag:
            results['normalized'] = normalize_text(results['regularized'])

        # 3. 토큰화
        results['tokenized'] = results['normalized'].split()

        # 4. 불용어 제거
        if self.remove_stopwords_flag:
            results['filtered'] = remove_stopwords(
                results['tokenized'],
                self.stopwords
            )
        else:
            results['filtered'] = results['tokenized']

        return results

### 예제 5: TextPreprocessor 사용

In [ ]:
print("\n" + "=" * 80)
print("[예제 5] 통합 전처리 파이프라인")
print("=" * 80)

# Preprocessor 초기화
preprocessor = TextPreprocessor(
    stopwords=KOREAN_STOPWORDS,
    remove_stopwords_flag=True,
    normalize_flag=True,
    regularize_flag=True
)

# 테스트 텍스트
test_text = "2024년12월1일에 010-1234-5678로 연락주세요. 가격은 12,000원입니다."

print("\n원본 텍스트:")
print("  \"%s\"" % test_text)

# 전처리 실행
tokens = preprocessor.preprocess(test_text)

print("\n최종 토큰:")
print("  %s" % tokens)


[예제 5] 통합 전처리 파이프라인

원본 텍스트:
  "2024년12월1일에 010-1234-5678로 연락주세요. 가격은 12,000원입니다."

최종 토큰:
  ['date에', 'phone로', '연락주세요.', '가격은', 'num원입니다.']


---

## 6. 의료 판독문 전처리 실습

### 의료 도메인 특화 불용어

In [ ]:
# 의료 도메인 불용어 확장
MEDICAL_STOPWORDS = KOREAN_STOPWORDS.union(set([
    # 의료 불용어
    '소견', '상', '님', '환자', '분', '씨',
    '있습니다', '없습니다', '보입니다', '됩니다',
    '합니다', '하십시오', '하세요',

    # 위치 표현
    '상부', '하부', '중앙', '전체', '부분', '영역',

    # 정도 표현
    '경미한', '중등도', '심한', '약간', '다소', '상당한'
]))

print("=" * 80)
print("의료 도메인 불용어 통계")
print("=" * 80)
print("\n일반 한국어 불용어: %d개" % len(KOREAN_STOPWORDS))
print("의료 도메인 불용어: %d개" % len(MEDICAL_STOPWORDS))
print("추가된 불용어: %d개" % (len(MEDICAL_STOPWORDS) - len(KOREAN_STOPWORDS)))

의료 도메인 불용어 통계

일반 한국어 불용어: 62개
의료 도메인 불용어: 86개
추가된 불용어: 24개


### 예제 6: 의료 판독문 전처리

In [ ]:
print("\n" + "=" * 80)
print("[예제 6] 의료 판독문 전처리")
print("=" * 80)

# 의료 전용 Preprocessor
medical_preprocessor = TextPreprocessor(
    stopwords=MEDICAL_STOPWORDS,
    remove_stopwords_flag=True,
    normalize_flag=True,
    regularize_flag=True
)

# 샘플 판독문
medical_reports = [
    "양측 폐야는 깨끗합니다. 심비대 소견은 없습니다.",
    "우상엽에 3.2cm 크기의 결절이 관찰됩니다.",
    "2024-11-15 CT 검사 결과: 간에 1.5cm 음영 보임",
    "환자분은 010-1234-5678로 연락 주세요. 재검사 날짜는 12월 20일입니다."
]

for i, report in enumerate(medical_reports, 1):
    print("\n[판독문 %d]" % i)

    results = medical_preprocessor.preprocess_full(report)

    print("원본:")
    print("  \"%s\"" % results['original'])

    print("정규화:")
    print("  \"%s\"" % results['regularized'])

    print("표준화:")
    print("  \"%s\"" % results['normalized'])

    print("토큰화:")
    print("  %s" % results['tokenized'])

    print("최종 (불용어 제거):")
    print("  %s" % results['filtered'])

    print("감소율: %.1f%% (%d → %d 토큰)" % (
        100.0 * (len(results['tokenized']) - len(results['filtered'])) / len(results['tokenized']),
        len(results['tokenized']),
        len(results['filtered'])
    ))


[예제 6] 의료 판독문 전처리

[판독문 1]
원본:
  "양측 폐야는 깨끗합니다. 심비대 소견은 없습니다."
정규화:
  "양측 폐야는 깨끗합니다. 심비대 소견은 없습니다."
표준화:
  "양측 폐야는 깨끗합니다. 심비대 소견은 없습니다."
토큰화:
  ['양측', '폐야는', '깨끗합니다.', '심비대', '소견은', '없습니다.']
최종 (불용어 제거):
  ['양측', '폐야는', '깨끗합니다.', '심비대', '소견은', '없습니다.']
감소율: 0.0% (6 → 6 토큰)

[판독문 2]
원본:
  "우상엽에 3.2cm 크기의 결절이 관찰됩니다."
정규화:
  "우상엽에 NUMcm 크기의 결절이 관찰됩니다."
표준화:
  "우상엽에 numcm 크기의 결절이 관찰됩니다."
토큰화:
  ['우상엽에', 'numcm', '크기의', '결절이', '관찰됩니다.']
최종 (불용어 제거):
  ['우상엽에', 'numcm', '크기의', '결절이', '관찰됩니다.']
감소율: 0.0% (5 → 5 토큰)

[판독문 3]
원본:
  "2024-11-15 CT 검사 결과: 간에 1.5cm 음영 보임"
정규화:
  "DATE CT 검사 결과: 간에 NUMcm 음영 보임"
표준화:
  "date ct 검사 결과: 간에 numcm 음영 보임"
토큰화:
  ['date', 'ct', '검사', '결과:', '간에', 'numcm', '음영', '보임']
최종 (불용어 제거):
  ['date', 'ct', '검사', '결과:', '간에', 'numcm', '음영', '보임']
감소율: 0.0% (8 → 8 토큰)

[판독문 4]
원본:
  "환자분은 010-1234-5678로 연락 주세요. 재검사 날짜는 12월 20일입니다."
정규화:
  "환자분은 PHONE로 연락 주세요. 재검사 날짜는 NUM월 NUM일입니다."
표준화:
  "환자분은 phone로 연락 주세요. 재검사 날짜는 num월 num일입니다."
토큰화:
  ['환자분은', 'phone로', '연락', '주

## 7. Summary

본 핸즈온 실습에서는 텍스트 전처리 파이프라인 구축을 목표로 다음과 같은 주요 기법들을 학습하고 구현했습니다.

1.  **불용어(Stopwords) 제거**: 텍스트 분석에서 의미 없는 단어(조사, 어미, 관사 등)를 제거하여 데이터의 노이즈를 줄이고 핵심 의미에 집중할 수 있도록 했습니다. 한국어 및 영어 불용어 리스트를 예시로 `remove_stopwords` 함수를 구현하여 적용했습니다.

2.  **표준화(Normalization)**: 동일한 의미를 가진 다양한 텍스트 형태를 통일된 형태로 변환했습니다. 이를 위해 `normalize_text` 함수를 통해 대소문자 통일, 반복 문자/구두점 축약, 다중 공백 처리 등의 기법을 적용했습니다.

3.  **정규화(Regularization)**: 특정 패턴(숫자, 날짜, 전화번호 등)을 대표 토큰(NUM, DATE, PHONE)으로 치환하여 Vocabulary 크기를 줄이고 모델의 일반화 성능을 향상시켰습니다. `normalize_numbers`, `normalize_dates`, `normalize_phone_numbers` 함수를 구현하여 순서대로 적용하는 `normalize_patterns`를 만들었습니다.

4.  **통합 전처리 파이프라인**: 위에서 구현한 개별 전처리 단계들을 `TextPreprocessor` 클래스 하나로 통합했습니다. 이 클래스는 불용어 제거, 표준화, 정규화 적용 여부를 선택적으로 설정할 수 있도록 설계되었으며, `preprocess` 및 `preprocess_full` 메서드를 제공합니다.

5.  **의료 판독문 전처리 실습**: 마지막으로 실제 의료 도메인의 특성을 반영하여 일반 한국어 불용어 리스트를 확장한 `MEDICAL_STOPWORDS`를 정의하고, `TextPreprocessor`를 활용하여 샘플 의료 판독문을 전처리하는 과정을 실습했습니다. 이 과정을 통해 각 전처리 단계가 텍스트에 어떻게 적용되고 최종 결과에 어떤 영향을 미치는지 확인했습니다.

이 실습을 통해 텍스트 데이터의 품질을 향상시키고, 이후 텍스트 분석 및 자연어 처리 모델링에 적합한 형태로 데이터를 가공하는 핵심 역량을 습득할 수 있었습니다.

---